# ChatGPT Demo

用于展示 OpenAI 的 Chat-API

## 1. 准备工作

1. 安装：Python 3.7+
2. 安装 / 升级 Python 库: `openai`, `langchain`
    - 命令行输入：`pip3 install --upgrade openai langchain`
3. 设置 API Key: 放到 环境变量 `OPENAI_API_KEY` 里；

**注：** `OpenAI` 要绑 信用卡 才能使用 API，见 [这里](./00_setup.md)

In [4]:
import sys

print(f"python 版本 不能小于 3.7.0，当前版本为 {sys.version}")

python 版本 不能小于 3.7.0，当前版本为 3.10.10 | packaged by Anaconda, Inc. | (main, Mar 21 2023, 18:39:17) [MSC v.1916 64 bit (AMD64)]


In [1]:
import openai

# 确认版本, openai 必须在 0.27.7 或以上
print("openai 版本不能小于 0.27.7, 目前版本为: ", openai.__version__)

openai 版本不能小于 0.27.7，目前版本为:  0.27.7


In [2]:
import langchain

# 确认版本, langchain 必须在 0.0.188 或以上
print("langchain 版本不能小于 0.0.188, 目前版本为: ", langchain.__version__)

langchain 版本不能小于 0.0.188, 目前版本为:  0.0.190


In [2]:
import os

# 记得安装openai package: pip3 install openai
import openai

# 事先将 OpenAI的 API-Key 存到 环境变量 'OPENAI_API_KEY' 里面
openai.api_key = os.getenv('OPENAI_API_KEY')

## 2. API 初步

### 2.1. role

不管是 问问题，还是 gpt的回答，都有个 `role` 字段

根据文档，目前 `role` 的值 有三个，分别代表三个角色：

+ `user` 代表 用户角色
+ `assistant` 代表 机器助手 角色
+ `system` 代表 系统，就是 环境方面的提示

In [3]:
# 一条记录，这里是由用户询问的问题
message = {
    "role": "user",
    "content": "你好，请说出一首唐诗"
}

# ChatCompletion 是对话模型；
response = openai.ChatCompletion.create(
    # 如果帐号申请到了 gpt-4, 这里可以填 "gpt-4"
    model="gpt-3.5-turbo",
    
    # 这里填的是对话列表
    messages=[message],
)

# 这里可以看到，回复是一个json对象，其中最重要的字段是它回复的答案，在这里：
# response["choices"] = [{
#   "message": {
#         "content": 回复的文本在这里
#         "role": "assistant"
#     }
# }]
answer = response['choices'][0]['message']['content']

print(f"回复: {answer}")

回复: 《春望》 - 唐代王之涣

国破山河在，
城春草木深。
感时花溅泪，
恨别鸟惊心。
烽火连三月，
家书抵万金。
白头搔更短，
浑欲不胜簪。


## 3. `Memory` 记忆

因为 参数 `messages` 是个 列表 []，所以这里可以放多条记录；

例子：可以和GPT聊天的历史记录串起来，形成一个有上下文的会话；

这个历史记录，在 LangChain 里面成为 `Memory` (记忆)

**注：** OpenAI 的 Chat API `没有状态`，每次通信都是独立的。

定义 记忆：

In [14]:
# Memory 记忆，代表对话历史
memory = []


下面将一次问答封装成函数的形式：

In [20]:
def simple_answer(question):
    """ 将 一问一答 封装成 函数，方便调用

    注：这里没有处理各种异常情况，毕竟只是演示；
    
    """

    # 一条记录，这里是由用户询问的问题
    message = {
        "role": "user",
        "content": question
    }

    # 将问题加入记忆
    memory.append(message)

    # ChatCompletion 是对话模型；
    response = openai.ChatCompletion.create(
        # 如果帐号申请到了 gpt-4, 这里可以填 "gpt-4"
        model="gpt-3.5-turbo",
        
        # 这里填的是 记忆
        messages=memory,
    )

    reply = response['choices'][0]['message']

    # 将回复加入记忆
    memory.append({
        "content": reply["content"],
        "role": reply["role"],
    })

    answer = reply['content']

    print(f"问题：{question}")
    print(f"回复: {answer}")
    print("")

调用函数，形成对话：

In [15]:
simple_answer("你好，请说出一首唐诗")

simple_answer("它的作者是谁？有什么贡献？")

问题：你好，请说出一首唐诗
回复: 《登鹳雀楼》
白日依山尽，黄河入海流。
欲窮千里目，更上一層樓。

问题：它的作者是谁？有什么贡献？
回复: 这首唐诗的作者是唐代著名诗人王之涣，他是唐代文学的代表人物之一，与杜甫、李白等被誉为“唐诗三杰”。王之涣的诗歌风格清新脱俗，富有哲理性和浪漫主义情怀，被后人称颂为“王家之风”。     

《登鹳雀楼》是王之涣的代表作之一，这首诗主要描写了作者登上高楼远眺，眼界开阔，心情豁然，力主不断追求更高的境界。这首诗不仅表现了王之涣的历史性思考和人文主义情怀，同时也打破了唐诗以往平淡无奇的风格，开拓了唐诗的新领域，具有重要的艺术和时代意义。



## 4. 其他参数

上面的接口 除了 model 和 messages，还有一些有用的参数，下面展开：

|参数名|说明|
|--|--|
|`model`|chat-模型，比如 "gpt-3.5-turbo", "gpt-4"|
|`messages`|对话列表|
|`max_tokens`|最大token数；对 3.5来说，该值不能超过 4K；|
|`temperature`|温度 取值0到2的小数，数值越大，回复约随机|
|`n`|该api回复问题的个数，写5，那么回复的messages就有5个候选答案|

### 4.1. 关于 `token`

+ 1 token 大概是 0.75个英文单词；
+ 1个中文字 大概是 2个英文单词；

上限：

+ "gpt-3.5-turbo" 4K = 4096 tokens，包含 问答
+ "gpt-4" 8K = 8192 tokens
+ "gpt-4-32" 32K，目前 很难申请到


下面，将相关的参数封装成一个函数：

In [21]:

memory = []

def get_answer(question, debug=False, model="gpt-3.5-turbo", n=1, max_tokens=1000, temperature=0.5):
    """ 用 ChatGPT 回答问题

    Args:
        question (str): 问题
        model (str, optional): 模型. Defaults to "gpt-3.5-turbo". 可以填 "gpt-4"
        n (int, optional): 生成多少个候选答案. Defaults to 1.
        max_tokens (int, optional): 限制不能超过多少个token. Defaults to 1000.
        temperature (float, optional): 温度，值越大越随机，[0, 2] Defaults to 0.5.
    """

    # 输入的每个元素，都是一个 对象
    # "role" 有三种type: user, assistant, system
    #     user 代表 用户输入的问题
    #     assistant 代表 机器人回答的问题
    #     system 代表 系统提示
    record = {
        "role": "user",
        "content": question
    }

    # 历史 多加一条问题
    memory.append(record)

    if debug:
        print(f"该次的输入参数: {memory}")

    response = openai.ChatCompletion.create(
        # 必须
        model=model,
        # 这里填的是所有的历史
        messages=memory,

        # Optional
        n=n,
        max_tokens=max_tokens,
        temperature=temperature,

        presence_penalty=0,
        frequency_penalty=0,
        stream=False, # 是否用流式接口一个字一个字的返回
    )

    reply = response["choices"][0]["message"]
    
    # 历史 多加一条答案
    memory.append({
        "content": reply["content"],
        "role": reply["role"],
    })

    return reply["content"]

In [22]:
memory = []

question = "你好，请说出一首唐诗"
answer = get_answer(question, debug=True)

print("")
print(f"问题：{question}")
print(f"回复: {answer}")
print("")

question = "请问它的作者是谁？"
answer = get_answer(question, debug=True)

print("")
print(f"问题：{question}")
print(f"回复: {answer}")
print("")



该次的输入参数: [{'role': 'user', 'content': '你好，请说出一首唐诗'}]

问题：你好，请说出一首唐诗
回复: 《登高》

风急天高猿啸哀，渚清沙白鸟飞回。
无边落木萧萧下，不尽长江滚滚来。
万里悲秋常作客，百年多病独登台。
艰难苦恨繁霜鬓，潦倒新停浊酒杯。

该次的输入参数: [{'role': 'user', 'content': '你好，请说出一首唐诗'}, {'content': '《登高》\n\n风急天高猿啸哀，渚清沙白鸟飞回。\n无边落木萧萧下，不尽长江滚滚来。\n万里悲秋常作客，百年多病独登台。\n艰难苦恨繁霜鬓，潦倒新停浊酒杯。', 'role': 'assistant'}, {'role': 'user', 'content': '请问它的作者是谁？'}]

问题：请问它的作者是谁？
回复: 这首诗的作者是唐代诗人杜甫。



## 5. 再说 `role`

经常听到 下面三个术语：

+ `zero-shot prompt` 0-例子 提示
+ `one-shot prompt`  1-例子 提示
+ `few-shot prompt`  少数-例子 提示

可以 通过 messages 的 role 完成的，完全可以在第一个问题之前，`伪造` 一些 机器回复的例子；

这些概念，在 `LangChain`，被封装在 `Prompt Template` 的模块中


In [24]:
# 我们是上帝，为机器制造 先天记忆 ！
memory = [{
    "role": "system",
    "content": "这里，需要你扮演唐诗引导者的角色，对玩家的问题，要回答 你好，这是唐诗 ***，请欣赏: ***"
}, {
    "role": "user",
    "content": "你好，请说出一首唐诗"
}, {
    "role": "assistant",
    "content": '你好，这是唐诗《春晓》，请欣赏:\n春眠不觉晓，处处闻啼鸟。\n夜来风雨声，花落知多少。'
}]

In [25]:
question = "你好，请说出一首唐诗"
answer = get_answer(question, debug=True)

print("")
print(f"问题：{question}")
print(f"回复: {answer}")
print("")

question = "请问它的作者是谁？"
answer = get_answer(question, debug=True)

print("")
print(f"问题：{question}")
print(f"回复: {answer}")
print("")


该次的输入参数: [{'role': 'system', 'content': '这里，需要你扮演唐诗引导者的角色，对玩家的问题，要回答 你好，这是唐诗 ***，请欣赏: ***'}, {'role': 'user', 'content': '你好，请说出一首唐诗'}, {'role': 'assistant', 'content': '你好，这是唐诗《春晓》，请欣赏:\n春眠不觉晓，处处闻啼鸟。\n夜来风雨声，花落知多少。'}, {'role': 'user', 'content': '你好，请说出一首唐诗'}]

问题：你好，请说出一首唐诗
回复: 你好，这是唐诗《登高》，请欣赏：
风急天高猿啸哀，渚清沙白鸟飞回。
无边落木萧萧下，不尽长江滚滚来。

该次的输入参数: [{'role': 'system', 'content': '这里，需要你扮演唐诗引导者的角色，对玩家的问题，要回答 你好，这是唐诗 ***，请欣赏: ***'}, {'role': 'user', 'content': '你好，请说出一首唐诗'}, {'role': 'assistant', 'content': '你好，这是唐诗《春晓》，请欣赏:\n春眠不觉晓，处处闻啼鸟。\n夜来风雨声，花落知多少。'}, {'role': 'user', 'content': '你好，请说出一首唐诗'}, {'content': '你好，这是唐诗《登高》，请欣赏：\n风急天高猿啸哀，渚清沙白鸟飞回。\n无边落木萧萧下，不尽长江滚滚来。', 'role': 'assistant'}, {'role': 'user', 'content': '请问它的作者是谁？'}]

问题：请问它的作者是谁？
回复: 这首诗的作者是唐代著名诗人杜甫。



## 6. 特点 & 问题

### 6.1. 无状态

需要 每次都要带 上下文（包括 对话历史 和 先验知识）

如果你要为一本书回答问题，那么先验知识可能就是这本书的所有字符。

因为API是按token收费的，这意味着几个问题之后，以后的每个问题都会按照最大token数收费了，在gpt-4单价昂贵的场景，是不利的。


### 6.2. token 上限

对 Turbo 3.5 模型，token上限是 4k = 4096个token

复杂点的应用场景，既有历史记录，又有先验知识

这意味着 连续为一本书问相关的几个问题，不管是前者还是后者，都放不下了。

所以要引入 `嵌入式向量`，将`文本分块`，建`索引`，并将索引放到 `向量数据库`；每次问问题，就到数据库取和问题最相关的几条记录，并上最近的几个历史，传到 api 获得答案；

这里：如果历史记录太多，那么历史记录本身就要建立索引；

`LangChain` 的 `Index` 组件，封装了这个流程；



### 6.3. 通信限制

OpenAI对 每个账号，有通信频率限制，不管是每分钟，还是每天总量都有限制，所以需要`调度`；

`调度`策略，目前需要自己处理，但和LangChain 功能类似的 [`Simantic Kernel`](https://github.com/microsoft/semantic-kernel) 目前貌似封装了这个流程；



## 7. LangChain 概念一句话概括

|组件|功能|其他|
|--|--|--|
|`Model`|封装 模型和参数||
|`Prompt`|封装 提示模板，示例||
|`Index`|封装 索引，存储||
|`Memory`|封装 历史记录||
|`Chain`|将上面各组件串成一个整体，方便使用||
|`Agent`|智能体，和插件功能类似，见下文分解||